In [48]:
# Importe Librerias
import calendar
import sys
import os
import psycopg2

from datetime import datetime, timedelta

print("Librerias Importadas")

Librerias Importadas


In [49]:
# Conexión a bd
sys.path.append(os.path.abspath(os.getcwd() + "/.."))

from db_conexion import get_spark_session, write_table, get_postgres_connection

spark = get_spark_session()

print(spark.conf.get("spark.driver.extraClassPath"))

def execute_postgres_query(query):
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="postgres",
        host="localhost"
    )
    cur = conn.cursor()
    
    try:
        cur.execute(query)
        conn.commit()
        print("Sentencia ejecutada correctamente.")
    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
    finally:
        cur.close()
        conn.close()


print("Función creada")


/workspaces/gd-test-bigdata/notebooks/EJERCICIO-1/postgresql-42.7.5.jar
Función creada


In [50]:
# Función verificar si tabla existe
def table_exists(spark, table_name):
    url, properties = get_postgres_connection()
    
    df = spark.read.jdbc(url=url, table="information_schema.tables", properties=properties)

    exists = df.filter(df["table_name"] == table_name).count() > 0

    return exists

In [51]:
# Creación de tablas con claves foráneas
ddl_statements = {
    "clientes": """CREATE TABLE clientes (
        id BIGSERIAL PRIMARY KEY,
        nombre VARCHAR(100) NOT NULL,
        email VARCHAR(100) UNIQUE NOT NULL,
        telefono VARCHAR(20)
    );""",
    "productos": """CREATE TABLE productos (
        id BIGSERIAL PRIMARY KEY,
        nombre VARCHAR(100) NOT NULL,
        descripcion TEXT,
        precio NUMERIC(10,5) NOT NULL
    );""",
    "corresponsales": """CREATE TABLE corresponsales (
        id BIGSERIAL PRIMARY KEY,
        nombre VARCHAR(255) NOT NULL,
        direccion VARCHAR(255),
        telefono VARCHAR(20),
        email VARCHAR(255),
        fecha DATE DEFAULT CURRENT_DATE
    );""",
    "ordenes": """CREATE TABLE ordenes (
        id BIGSERIAL,
        fecha DATE DEFAULT CURRENT_DATE,
        id_cliente BIGINT REFERENCES clientes(id) ON DELETE CASCADE,
        id_corresponsal BIGINT REFERENCES corresponsales(id) ON DELETE SET NULL,
        PRIMARY KEY (id, fecha)
    ) PARTITION BY RANGE (fecha);
    """,
    "detalles_ordenes": """CREATE TABLE detalles_ordenes (
        id BIGSERIAL PRIMARY KEY,
        id_orden BIGINT,
        fecha DATE,
        id_producto BIGINT REFERENCES productos(id) ON DELETE CASCADE,
        cantidad INT NOT NULL,
        precio NUMERIC(10,5) NOT NULL,
        FOREIGN KEY (id_orden, fecha) REFERENCES ordenes(id, fecha) ON DELETE CASCADE
    );"""
}

# Ejecutar los comandos SQL solo si la tabla no existe
for table, ddl in ddl_statements.items():
    if not table_exists(spark, table):
        execute_postgres_query(ddl)

        print(f"Tabla '{table}' creada correctamente.")
    else:
        print(f"Tabla '{table}' ya existe, saltando creación.")


print("Proceso de creación de tablas completado.")

Sentencia ejecutada correctamente.
Tabla 'clientes' creada correctamente.
Sentencia ejecutada correctamente.
Tabla 'productos' creada correctamente.
Sentencia ejecutada correctamente.
Tabla 'corresponsales' creada correctamente.
Sentencia ejecutada correctamente.
Tabla 'ordenes' creada correctamente.
Sentencia ejecutada correctamente.
Tabla 'detalles_ordenes' creada correctamente.
Proceso de creación de tablas completado.
